## Upload Retrospective Forecasts to `cdc-flusight` Zoltar Project

In [1]:
# import libraries
import glob
import pymmwr as pm
from pathlib import Path
import ntpath
import time
import pandas as pd
from zoltpy import util
import os
import datetime
import yaml

In [2]:
# initialize variables
my_path = "../../model-forecasts/component-models"
directory = "master"
cdc_project_name = "CDC Retrospective Forecasts"

In [3]:
def get_available_timezeros(project, model):
    model_timezeros = vars(model)['json']['forecasts']
    project_timezeros = vars(project)['json']['timezeros']
    empty_model_timezeros = []
    for i in range(len(project_timezeros)):
        timezero = project_timezeros[i]['timezero_date']
        if timezero not in model_timezeros:
            empty_model_timezeros += [timezero]
    return empty_model_timezeros

In [4]:
def get_timezero(ew_week, season_start_year):
    """
    :param ew_week: an epi week from within a cdc csv forecast file. e.g., 1, 30, 52
    :param season_start_year
    :return: a datetime.date that is the Monday of the EW corresponding to the args
    """
    if ew_week < 30:
        sunday_date = pm.mmwr_week_to_date(season_start_year + 1, ew_week)
    else:
        sunday_date = pm.mmwr_week_to_date(season_start_year, ew_week)
    return (sunday_date + datetime.timedelta(days=1)).strftime('%Y-%m-%d')


In [13]:
# for loop to find all csvs in forecasts
error_upload = []
start = time.time()
conn = util.authenticate()
project = [project for project in conn.projects if project.name == cdc_project_name][0]
models = [model for model in project.models]
model_names = [model.name for model in project.models]

# loop through model directories
for first_path in glob.iglob(my_path + "**/**/", recursive=False):
    
    # continue authentication
    conn = util.authenticate()
    project = [project for project in conn.projects if project.name == cdc_project_name][0]
    
    # initialize zoltar model name
    model_name = ''
    
    # get model directory
    path = os.path.dirname(first_path)
    curr_model_dir = os.path.basename(path)
    print("Model: ", curr_model_dir)
    
    # set up batch variables
    predx_batch = []
    forecast_filename_batch = []
    timezero_batch = []
    
#if curr_model_dir == 'CU_EAKFC_SEIRS':
    metadata = yaml.load(open(path + "/metadata.txt", 'r'))

    # loop through model forecasts
    for csv_file in glob.iglob(first_path + "*.csv", recursive=False):

        # get model timezero
        csv_file_name = ntpath.basename(csv_file)
        csv_file_list = csv_file_name.split("-")
        epi_week = int(csv_file_list[0][-2:])
        epi_year = int(csv_file_list[1])

        # Check if forecast is retrospective
        real_time_start_season = int(metadata["real_time_start_season"][:4])

        if epi_year < real_time_start_season:
            timezero = get_timezero(epi_week, epi_year)

            # get zoltar model name
            model_name = metadata["model_name"]
            model = [model for model in models if model.name == model_name][0]

            # check what timezeros are available
            avail_timezeros = get_available_timezeros(project, model)
            if timezero in avail_timezeros:
                print("Add model =", model_name, "timezero =", timezero)
                try:
                    conn.re_authenticate_if_necessary()
                    predx_json, forecast_filename = util.convert_cdc_csv_to_json_io_dict(epi_year, csv_file)
                    predx_batch += [predx_json]
                    forecast_filename_batch += [forecast_filename]
                    timezero_batch += [timezero]
                except:
                    print("ERROR")
                    error_upload += [csv_file + ", "] 
    if model_name in model_names and len(predx_batch) != 0:
        print('\n\nUPLOADING BATCH OF FORECASTS')
        conn = util.authenticate()
        try:
            util.upload_forecast_batch(conn, predx_batch, forecast_filename_batch, 
                               cdc_project_name, model_name, timezero_batch, overwrite=False)
        except Exception as e: 
            print(e)

print("\n\nRUN COMPLETE")
end = time.time()
interval = end - start
print("Total minutes elapsed: ", interval / 60.0)
print("ERROR UPLOADING...", error_upload)
print("\n".join(map(str, error_upload)))

Model:  Protea_Kudu
Add model = Subtype weighted historical average model timezero = 2017-05-08


/Users/katiehouse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


Add model = Subtype weighted historical average model timezero = 2012-11-26
Add model = Subtype weighted historical average model timezero = 2013-02-25
Add model = Subtype weighted historical average model timezero = 2015-02-23
Add model = Subtype weighted historical average model timezero = 2014-12-01
Add model = Subtype weighted historical average model timezero = 2015-12-14
Add model = Subtype weighted historical average model timezero = 2017-10-30
Add model = Subtype weighted historical average model timezero = 2013-01-14
Add model = Subtype weighted historical average model timezero = 2014-11-10
Add model = Subtype weighted historical average model timezero = 2012-01-02
Add model = Subtype weighted historical average model timezero = 2016-12-26
Add model = Subtype weighted historical average model timezero = 2014-04-07
Add model = Subtype weighted historical average model timezero = 2016-03-14
Add model = Subtype weighted historical average model timezero = 2012-10-15
Add model = 

Add model = Subtype weighted historical average model timezero = 2017-12-25
Add model = Subtype weighted historical average model timezero = 2013-04-08
Add model = Subtype weighted historical average model timezero = 2012-12-17
Add model = Subtype weighted historical average model timezero = 2018-04-16
Add model = Subtype weighted historical average model timezero = 2010-11-08
Add model = Subtype weighted historical average model timezero = 2016-01-11
Add model = Subtype weighted historical average model timezero = 2013-11-18
Add model = Subtype weighted historical average model timezero = 2015-04-06
Add model = Subtype weighted historical average model timezero = 2013-03-04
Add model = Subtype weighted historical average model timezero = 2015-10-26
Add model = Subtype weighted historical average model timezero = 2016-10-10
Add model = Subtype weighted historical average model timezero = 2014-02-03
Add model = Subtype weighted historical average model timezero = 2012-03-19
Add model = 

Add model = Subtype weighted historical average model timezero = 2017-10-16
Add model = Subtype weighted historical average model timezero = 2013-01-28
Add model = Subtype weighted historical average model timezero = 2011-11-14
Add model = Subtype weighted historical average model timezero = 2015-01-05
Add model = Subtype weighted historical average model timezero = 2013-12-23
Add model = Subtype weighted historical average model timezero = 2017-04-10
Add model = Subtype weighted historical average model timezero = 2010-12-13
Add model = Subtype weighted historical average model timezero = 2016-04-25
Add model = Subtype weighted historical average model timezero = 2012-10-29
Add model = Subtype weighted historical average model timezero = 2018-01-15
Add model = Subtype weighted historical average model timezero = 2017-11-20
Add model = Subtype weighted historical average model timezero = 2016-12-19
Add model = Subtype weighted historical average model timezero = 2014-04-14
Add model = 

...

Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2012-12-24
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2015-04-27
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2016-05-09
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2014-04-28
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2013-04-15
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2014-12-29
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2017-04-10
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2014-03-17
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2010-12-13
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2015-04-06
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2012-03-26
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2012-12-17
Add model = Ensemble Adjustment Kalman Filter SIRS timezero = 2012-05-07
Add model = Ensemble Adjustment Kalman Filter SIRS 

upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2017-02-06 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2015-11-02 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2013-12-30 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2010-10-25 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2016-10-17 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2012-02-27 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2013-11-04 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adju

upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2014-10-13 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2011-11-07 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2014-02-24 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2013-02-11 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2010-10-11 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2014-01-13 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2015-11-16 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adju

upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2012-03-26 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2012-12-17 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2012-05-07 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2014-05-12 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2016-03-07 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2016-12-26 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2014-03-10 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adju

upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2016-04-11 timezero...
upload complete
uploading CDC Retrospective Forecasts project, Ensemble Adjustment Kalman Filter SIRS model, 2013-12-09 timezero...
upload complete

* polling for status change. upload_file_job: ('UploadFileJob', 'https://zoltardata.com/api/uploadfilejob/8494/', 8494, 'QUEUED')
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- QUEUED
- SUCCESS


RUN COMPLETE
Total minutes elapsed:  116.1179807027181
ERROR UPLOADING... ['../../model-forecasts/component-models/LANL_DBMplus/EW19-2011-LANL_DBMplus.csv, ', '../../model-forecasts/component-models/LANL_DBMplus/EW19-2015-LANL_DBMplus.csv, ', '../../model-forecasts/component-models/LANL_DBMplus/EW20-2013-LANL_DBMplus.csv, ', '../../model-forecasts/component-models/LANL_DBMplus/EW20-2016-LANL_DBMplus.csv, ', '../../model-forecasts

## conn = util.authenticate()
project = [project for project in conn.projects if project.name == cdc_project_name][0]
model_name = 'Subtype weighted historical average model'
model = [model for model in models if model.name == model_name][0]


In [8]:
model_timezeros = vars(model)['json']['forecasts']

In [11]:
vars(model)['json']['forecasts'][0]

'https://zoltardata.com/api/forecast/2874/'